In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

In [4]:
# step 1. cleaning data
cosm = pd.read_csv('datasets/cosmetic.csv')
cosm.info()

cosm = cosm.loc[pd.notnull(cosm['ingredients'])]
cosm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1630 entries, 0 to 1629
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Label        1630 non-null   object 
 1   URL          1630 non-null   object 
 2   brand        1626 non-null   object 
 3   name         1626 non-null   object 
 4   price        1626 non-null   object 
 5   rank         1624 non-null   float64
 6   skin_type    1626 non-null   object 
 7   ingredients  1625 non-null   object 
dtypes: float64(1), object(7)
memory usage: 102.0+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1625 entries, 0 to 1629
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Label        1625 non-null   object 
 1   URL          1625 non-null   object 
 2   brand        1625 non-null   object 
 3   name         1625 non-null   object 
 4   price        1625 non-null   object 
 5   rank         16

In [5]:
# label
cosm.Label[cosm['Label'] == 'moisturizing-cream-oils-mists'] = str('Moisturizer')
cosm.Label[cosm['Label'] == 'cleanser'] = str('Cleanser')
cosm.Label[cosm['Label'] == 'facial-treatments'] = str('Treatment')
cosm.Label[cosm['Label'] == 'face-mask'] = str('Face Mask')
cosm.Label[cosm['Label'] == 'eye-treatment-dark-circle-treatment'] = str('Eye cream')
cosm.Label[cosm['Label'] == 'sunscreen-sun-protection'] = str('Sun protect')

C:\Users\SAISAR~1.YEC\AppData\Local\Temp/ipykernel_15648/2748207914.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cosm.Label[cosm['Label'] == 'cleanser'] = str('Cleanser')
C:\Users\SAISAR~1.YEC\AppData\Local\Temp/ipykernel_15648/2748207914.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cosm.Label[cosm['Label'] == 'facial-treatments'] = str('Treatment')
C:\Users\SAISAR~1.YEC\AppData\Local\Temp/ipykernel_15648/2748207914.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

In [6]:
# name -> duplicated item
df_2 = cosm['name'].drop_duplicates()
cosm = cosm.loc[df_2.index, :].reset_index()

In [7]:
# URL
cosm.drop(['URL', 'index'], axis = 1, inplace = True)

In [8]:
# price
pattern = re.compile(r"(\d+).\d+")
for i in range(len(cosm)):
    cosm['price'][i] = re.findall(pattern, cosm['price'][i])[0]

cosm['price'] = pd.to_numeric(cosm['price'])

C:\Users\SAISAR~1.YEC\AppData\Local\Temp/ipykernel_15648/3674295657.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cosm['price'][i] = re.findall(pattern, cosm['price'][i])[0]


In [9]:
# rank
cosm['rank'].fillna(0, inplace = True)
cosm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1472 entries, 0 to 1471
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Label        1472 non-null   object 
 1   brand        1472 non-null   object 
 2   name         1472 non-null   object 
 3   price        1472 non-null   int64  
 4   rank         1472 non-null   float64
 5   skin_type    1472 non-null   object 
 6   ingredients  1472 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 80.6+ KB


In [10]:
# skin_type
pattern = re.compile(r"([a-zA-Z]+)\\n")
for i in range(len(cosm)):
    cosm['skin_type'][i] = re.findall(pattern, cosm['skin_type'][i])

C:\Users\SAISAR~1.YEC\AppData\Local\Temp/ipykernel_15648/1062899077.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cosm['skin_type'][i] = re.findall(pattern, cosm['skin_type'][i])


In [11]:
# list column dummies
df_2 = cosm['skin_type'].str.join('|').str.get_dummies()
cosm_2 = cosm.join(df_2).drop('skin_type', axis = 1)

In [16]:
# tokenize ingredients
a = [t.split('\r\n\r\n') for t in cosm['ingredients']]
pattern = ['\r\n', '-\w+: ', 'Please', 'No Info', 'This product', 'Visit']

for i in range(len(cosm)):
    Num = len(a[i])
    for j in range(Num):
        if all(x not in a[i][j] for x in pattern):
           cosm_2['ingredients'][i] = a[i][j]

C:\Users\SAISAR~1.YEC\AppData\Local\Temp/ipykernel_15648/3001248997.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cosm_2['ingredients'][i] = a[i][j]


In [19]:
# save the file
cosm_2.to_csv('datasets/cosmetic_preprocessed1.csv', encoding = 'utf-8-sig', index = False)